In [1]:
import pandas as pd
import numpy as np
import time
from scipy.stats import norm
from scipy.optimize import minimize
from src.variable_change import a_calc_func, b_calc_func, w_calc_func, gamma_calc_func
from src.ML_estimation import log_likehood_variable_changed_fast
from src.sucess_probability import p_g

In [50]:
factor_loading_list = [0.45, 0.45, 0.45]
pd_list = [0.0015, 0.0100, 0.0500]
gamma_list = norm.ppf(pd_list)
num_of_obligors_list = [400, 250, 100]

In [51]:
def ml_param_estimation(
        default_table, num_of_obligors_table, factor_loading_init, gamma_list_init):
    # if len(factor_loading_init) == 1:
    #     factor_loading_init = np.full_like(gamma_list_init, factor_loading_init[0])
    # elif len(gamma_list_init) == 1:
    #     gamma_list_init = gamma_list_init * len(factor_loading_init)

    a_init = np.array(a_calc_func(np.array(factor_loading_init), np.array(gamma_list_init)))
    b_init = np.array(b_calc_func(np.array(factor_loading_init), np.array(gamma_list_init)))

    initial_guess = np.concatenate((a_init, b_init))

    num_of_a = len(a_init)
    bounds = [(-10, 10)] * len(initial_guess)

    # Optimization
    objective_function = lambda params: -log_likehood_variable_changed_fast(
        default_table, num_of_obligors_table, p_g, norm.pdf, params[:num_of_a], params[num_of_a:len(initial_guess)]
    )

    result = minimize(objective_function,
                      initial_guess,
                      method="Nelder-Mead",
                      bounds=bounds,
                      options={
                          'disp': False})

    factor_loading_result = np.array(w_calc_func(np.array(result.x[:num_of_a]), np.array(result.x[num_of_a:])))
    gamma_result = np.array(gamma_calc_func(np.array(result.x[:num_of_a]), np.array(result.x[num_of_a:])))

    return factor_loading_result, gamma_result, result

In [52]:
from src.data_generator import generate_default_time_series
years = 80
defaults_df = generate_default_time_series(factor_loading_list, num_of_obligors_list, gamma_list, years)
num_of_obligors_table = np.array([num_of_obligors_list] * len(defaults_df))

In [53]:
factor_loading_init = factor_loading_list
gamma_list_init = gamma_list
default_table = defaults_df.values

In [54]:
a_init = np.array(a_calc_func(np.array(factor_loading_init), np.array(gamma_list_init)))
b_init = np.array(b_calc_func(np.array(factor_loading_init), np.array(gamma_list_init)))

initial_guess = np.concatenate((a_init, b_init))

num_of_a = len(a_init)
bounds = [(-10, 10)] * len(initial_guess)

In [55]:
# Optimization
objective_function = lambda params: -log_likehood_variable_changed_fast(
    default_table, num_of_obligors_table, p_g, norm.pdf, params[:num_of_a], params[num_of_a:len(initial_guess)]
)

start_time = time.time()

result_nelder_mead = minimize(objective_function,
                  initial_guess,
                  method="Nelder-Mead",
                  bounds=bounds,
                  options={
                      'disp': False})

end_time = time.time()
time_nelder_mead = end_time - start_time

In [56]:
result_nelder_mead

       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 394.05198432619846
             x: [-5.884e-01 -5.481e-01 -4.997e-01 -3.446e+00 -2.598e+00
                 -1.838e+00]
           nit: 218
          nfev: 358
 final_simplex: (array([[-5.884e-01, -5.481e-01, ..., -2.598e+00,
                        -1.838e+00],
                       [-5.883e-01, -5.480e-01, ..., -2.598e+00,
                        -1.838e+00],
                       ...,
                       [-5.884e-01, -5.480e-01, ..., -2.599e+00,
                        -1.838e+00],
                       [-5.883e-01, -5.480e-01, ..., -2.598e+00,
                        -1.838e+00]]), array([ 3.941e+02,  3.941e+02,  3.941e+02,  3.941e+02,
                        3.941e+02,  3.941e+02,  3.941e+02]))

In [57]:
start_time = time.time()

result_BFGS = minimize(objective_function,
                  initial_guess,
                  method="BFGS",
                  bounds=bounds,
                  options={
                      'disp': False})

end_time = time.time()
time_BFGS = end_time - start_time

C:\Users\kissp\AppData\Local\Temp\ipykernel_8912\1320668608.py:3: RuntimeWarning: Method BFGS cannot handle bounds.
  result_BFGS = minimize(objective_function,


In [58]:
result_BFGS

  message: Desired error not necessarily achieved due to precision loss.
  success: False
   status: 2
      fun: 394.05198412269755
        x: [-5.884e-01 -5.480e-01 -4.997e-01 -3.446e+00 -2.598e+00
            -1.838e+00]
      nit: 14
      jac: [ 4.196e-05  1.144e-05 -3.815e-05 -2.289e-05  3.815e-06
             2.670e-05]
 hess_inv: [[ 3.315e-02  1.598e-02 ...  1.404e-02  1.060e-02]
            [ 1.598e-02  9.444e-03 ...  8.159e-03  5.624e-03]
            ...
            [ 1.404e-02  8.159e-03 ...  8.873e-03  5.586e-03]
            [ 1.060e-02  5.624e-03 ...  5.586e-03  5.319e-03]]
     nfev: 308
     njev: 44

In [72]:
try:
    result_LBFGSB = minimize(objective_function,
                      initial_guess,
                      method="L-BFGS-B",
                      bounds=bounds,
                      options={
                          'disp': False})
except Exception as e:
    print(e)

Error in function ibeta_derivative<d>(%1%,%1%,%1%): Overflow Error


In [60]:
start_time = time.time()
result_Powell = minimize(objective_function,
                  initial_guess,
                  method="Powell",
                  bounds=bounds,
                  options={
                      'disp': False})

end_time = time.time()
time_Powell = end_time - start_time

C:\Users\kissp\PycharmProjects\VasicekPDModell\src\ML_estimation.py:348: RuntimeWarning: divide by zero encountered in log
  return sum(np.log(calc_linear_likelihood(d_g_list, n_g_list, p_g, prob_dens_func, a, b)) for d_g_list, n_g_list in zip(d_g_array, n_g_array))
C:\Users\kissp\PycharmProjects\VasicekPDModell\venv\lib\site-packages\scipy\optimize\_optimize.py:2394: RuntimeWarning: invalid value encountered in scalar multiply
  q = (xf - fulc) * (fx - fnfc)


In [61]:
result_Powell

 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: 394.0527316794114
       x: [-5.873e-01 -5.465e-01 -4.992e-01 -3.446e+00 -2.599e+00
           -1.840e+00]
     nit: 3
   direc: [[ 0.000e+00  0.000e+00 ...  0.000e+00  1.000e+00]
           [ 0.000e+00  1.000e+00 ...  0.000e+00  0.000e+00]
           ...
           [ 0.000e+00  0.000e+00 ...  1.000e+00  0.000e+00]
           [-6.848e-02 -1.606e-03 ...  7.019e-03  2.503e-04]]
    nfev: 273

In [62]:
start_time = time.time()
result_COBLYA = minimize(objective_function,
                  initial_guess,
                  method="COBYLA",
                  bounds=bounds,
                  options={
                      'disp': False})

end_time = time.time()
time_COBYLA = end_time - start_time

In [63]:
result_COBLYA

 message: Optimization terminated successfully.
 success: True
  status: 1
     fun: 394.05217577162074
       x: [-5.869e-01 -5.476e-01 -4.995e-01 -3.444e+00 -2.598e+00
           -1.838e+00]
    nfev: 431
   maxcv: 0.0

In [64]:
start_time = time.time()
result_TNC = minimize(objective_function,
                  initial_guess,
                  method="TNC",
                  bounds=bounds,
                  options={
                      'disp': False})
end_time = time.time()
time_TNC = end_time - start_time

In [65]:
result_TNC

 message: Max. number of function evaluations reached
 success: False
  status: 3
     fun: 394.06895647385954
       x: [-5.760e-01 -5.471e-01 -4.979e-01 -3.424e+00 -2.593e+00
           -1.832e+00]
     nit: 14
     jac: [ 1.250e+00 -9.046e-01 -9.039e-02  6.553e-01 -1.933e-03
            8.793e-01]
    nfev: 707

In [66]:
# Save the results to a dataframe
# Columns: success, fun, nit, nfev, running_time, a0, a1, a2, b0, b1, b2
results_df = pd.DataFrame(columns=['success', 'fun', 'nit', 'nfev', 'running_time', 'a0', 'a1', 'a2', 'b0', 'b1', 'b2'])

results_df.loc['Nelder-Mead'] = [result_nelder_mead.success, result_nelder_mead.fun, result_nelder_mead.nit, result_nelder_mead.nfev, time_nelder_mead] + list(result_nelder_mead.x)
results_df.loc['BFGS'] = [result_BFGS.success, result_BFGS.fun, result_BFGS.nit, result_BFGS.nfev, time_BFGS] + list(result_BFGS.x)
#results_df.loc['L-BFGS-B'] = [result_LBFGSB.success, result_LBFGSB.fun, result_LBFGSB.nit, result_LBFGSB.nfev, time_BFGS] + list(result_LBFGSB.x)
results_df.loc['Powell'] = [result_Powell.success, result_Powell.fun, result_Powell.nit, result_Powell.nfev, time_Powell] + list(result_Powell.x)
results_df.loc['COBYLA'] = [result_COBLYA.success, result_COBLYA.fun, None, result_COBLYA.nfev, time_COBYLA] + list(result_COBLYA.x)
results_df.loc['TNC'] = [result_TNC.success, result_TNC.fun, result_TNC.nit, result_TNC.nfev, time_TNC] + list(result_TNC.x)


In [67]:
results_df

,success,fun,nit,nfev,running_time,a0,a1,a2,b0,b1,b2
Nelder-Mead,True,394.051984,218,358,63.706928,-0.588396,-0.548058,-0.499739,-3.445782,-2.598482,-1.838331
BFGS,False,394.051984,14,308,51.884348,-0.588375,-0.548029,-0.499718,-3.445768,-2.598484,-1.838329
Powell,True,394.052732,3,273,40.256153,-0.587327,-0.546506,-0.499202,-3.446437,-2.599097,-1.840031
COBYLA,True,394.052176,None,431,57.950325,-0.586910,-0.547602,-0.499507,-3.443587,-2.597629,-1.837777
TNC,False,394.068956,14,707,88.142456,-0.576034,-0.547098,-0.497858,-3.423949,-2.593108,-1.832005


In [68]:
years2 = 20
defaults_df2 = generate_default_time_series(factor_loading_list, num_of_obligors_list, gamma_list, years2)
num_of_obligors_table2 = np.array([num_of_obligors_list] * len(defaults_df2))

factor_loading_init = factor_loading_list
gamma_list_init = gamma_list
default_table = defaults_df2.values

a_init = np.array(a_calc_func(np.array(factor_loading_init), np.array(gamma_list_init)))
b_init = np.array(b_calc_func(np.array(factor_loading_init), np.array(gamma_list_init)))

initial_guess = np.concatenate((a_init, b_init))

num_of_a = len(a_init)
bounds = [(-10, 10)] * len(initial_guess)

In [70]:
start_time = time.time()
result_nelder_mead2 = minimize(objective_function,
                               initial_guess,
                              method="Nelder-Mead",
                              bounds=bounds,
                              options={
                                  'disp': False})
end_time = time.time()
time_nelder_mead2 = end_time - start_time

start_time = time.time()
result_BFGS2 = minimize(objective_function,
                        initial_guess,
                        method="BFGS",
                        bounds=bounds,
                        options={
                            'disp': False})
end_time = time.time()
time_BFGS2 = end_time - start_time

start_time = time.time()
result_Powell2 = minimize(objective_function,
                            initial_guess,
                            method="Powell",
                            bounds=bounds,
                            options={
                                'disp': False})
end_time = time.time()
time_Powell2 = end_time - start_time

start_time = time.time()
result_COBLYA2 = minimize(objective_function,
                            initial_guess,
                            method="COBYLA",
                            bounds=bounds,
                            options={
                                'disp': False})
end_time = time.time()
time_COBYLA2 = end_time - start_time

start_time = time.time()
result_TNC2 = minimize(objective_function,
                        initial_guess,
                        method="TNC",
                        bounds=bounds,
                        options={
                            'disp': False})
end_time = time.time()
time_TNC2 = end_time - start_time

# Save the results to a dataframe
# Columns: success, fun, nit, nfev, running_time, a0, a1, a2, b0, b1, b2
results_df2 = pd.DataFrame(columns=['success', 'fun', 'nit', 'nfev', 'running_time', 'a0', 'a1', 'a2', 'b0', 'b1', 'b2'])

results_df2.loc['Nelder-Mead'] = [result_nelder_mead2.success, result_nelder_mead2.fun, result_nelder_mead2.nit, result_nelder_mead2.nfev, time_nelder_mead2] + list(result_nelder_mead2.x)
results_df2.loc['BFGS'] = [result_BFGS2.success, result_BFGS2.fun, result_BFGS2.nit, result_BFGS2.nfev, time_BFGS2] + list(result_BFGS2.x)
results_df2.loc['Powell'] = [result_Powell2.success, result_Powell2.fun, result_Powell2.nit, result_Powell2.nfev, time_Powell2] + list(result_Powell2.x)
results_df2.loc['COBYLA'] = [result_COBLYA2.success, result_COBLYA2.fun, None, result_COBLYA2.nfev, time_COBYLA2] + list(result_COBLYA2.x)
results_df2.loc['TNC'] = [result_TNC2.success, result_TNC2.fun, result_TNC2.nit, result_TNC2.nfev, time_TNC2] + list(result_TNC2.x)

C:\Users\kissp\AppData\Local\Temp\ipykernel_8912\4219541889.py:12: RuntimeWarning: Method BFGS cannot handle bounds.
  result_BFGS2 = minimize(objective_function,
C:\Users\kissp\PycharmProjects\VasicekPDModell\src\ML_estimation.py:348: RuntimeWarning: divide by zero encountered in log
  return sum(np.log(calc_linear_likelihood(d_g_list, n_g_list, p_g, prob_dens_func, a, b)) for d_g_list, n_g_list in zip(d_g_array, n_g_array))
C:\Users\kissp\PycharmProjects\VasicekPDModell\venv\lib\site-packages\scipy\optimize\_optimize.py:2394: RuntimeWarning: invalid value encountered in scalar multiply
  q = (xf - fulc) * (fx - fnfc)
C:\Users\kissp\PycharmProjects\VasicekPDModell\venv\lib\site-packages\scipy\optimize\_optimize.py:2395: RuntimeWarning: invalid value encountered in scalar subtract
  p = (xf - fulc) * q - (xf - nfc) * r
C:\Users\kissp\PycharmProjects\VasicekPDModell\venv\lib\site-packages\scipy\optimize\_optimize.py:2396: RuntimeWarning: invalid value encountered in scalar subtract
  q 

In [71]:
results_df2

,success,fun,nit,nfev,running_time,a0,a1,a2,b0,b1,b2
Nelder-Mead,True,89.133810,230,357,13.166096,-0.479960,-0.667493,-0.492014,-3.377281,-2.807194,-1.978689
BFGS,True,89.133810,14,140,4.538379,-0.479963,-0.667500,-0.492028,-3.377245,-2.807212,-1.978695
Powell,True,89.141944,6,532,16.938418,-0.458966,-0.659624,-0.485254,-3.357195,-2.802342,-1.973534
COBYLA,True,89.133859,None,507,16.644754,-0.481421,-0.668247,-0.492427,-3.379167,-2.808410,-1.979234
TNC,False,89.136094,15,700,22.579412,-0.473681,-0.662060,-0.490537,-3.365148,-2.796606,-1.972487
